In [ ]:
# set tf 1.x for colab
%tensorflow_version 1.x

Read about ill-conditioning: http://cnl.salk.edu/~schraudo/teach/NNcourse/precond.html

In [1]:
import tensorflow as tf
import sys
sys.path.append("../..")
from keras_utils import reset_tf_session
s = reset_tf_session()
print("We're using TF", tf.__version__)
from matplotlib import animation, rc
import matplotlib.pyplot as plt
import matplotlib_utils
from IPython.display import HTML, display_html
import numpy as np

Using TensorFlow backend.


We're using TF 1.3.0


In [2]:
def plot_gd(x_scale, lr, steps):
    y_guess = tf.Variable([1.5, 1.5], dtype='float32')
    y_true = tf.constant([0.2, 0.2], dtype='float32')

    # x is x_scale times more important in loss (creates valleys)
    loss = tf.reduce_mean((tf.multiply(y_guess, tf.constant([x_scale, 1.])) - y_true)**2)

    step = tf.train.GradientDescentOptimizer(lr).minimize(loss, var_list=y_guess)
    
    # nice figure settings
    fig, ax = plt.subplots(figsize=(5, 5))
    y_true_value = s.run(y_true)
    level_x = np.arange(-2, 2, 0.05)
    level_y = np.arange(-2, 2, 0.05)
    X, Y = np.meshgrid(level_x, level_y)
    Z = (X * x_scale - y_true_value[0])**2 + (Y - y_true_value[1])**2
    ax.set_xlim(-2, 2)
    ax.set_ylim(-2, 2)
    s.run(tf.global_variables_initializer())
    ax.scatter(*s.run(y_true), c='red')
    contour = ax.contour(X, Y, Z, 10)
    ax.clabel(contour, inline=1, fontsize=10)
    line, = ax.plot([], [], lw=2)

    # start animation with empty trajectory
    def init():
        line.set_data([], [])
        return (line,)

    trajectory = [s.run(y_guess)]

    # one animation step (make one GD step)
    def animate(i):
        s.run(step)
        trajectory.append(s.run(y_guess))
        line.set_data(*zip(*trajectory))
        return (line,)

    anim = animation.FuncAnimation(fig, animate, init_func=init,
                                   frames=steps, interval=20, blit=True)

    anim.save(None, writer=matplotlib_utils.SimpleMovieWriter(0.0001))

In [3]:
plot_gd(x_scale=1.0, lr=0.1, steps=25)

<img src="%2BTl5ufo6erx8vP09fb3%2BPn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3%2BPn6/9oADAMBAAIRAxEAPwD9/KKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigBsssUETTzyKiIpZ3c4CgdST2Fcn8Jvj98CPj5ZX2pfAr41%2BEfGttplwINSuPCXiS11JLSUjIjla3kcRsQCdrYOK6m/lt4bGaa7haSJImaWNIWkLKAcgIoJckfwgEnoAa%2BTv2JfilafGD41eMf2ofGvwp%2BIXgvVPEun6T4V8NeDNd%2BEOv6U2k6HZ3F9JaPdzXFjHFJczS3VxLJ5btDaRNBEWDeZJKAfR/g742/Bn4ieL9d%2BHvw/wDi54Y13X/C04h8TaHo%2Bv291eaRISwCXUMbs9uxKsAJApyp9DXT18PfsKfD/wCIOl/GH4U%2BGNW%2BFHijQ9Q%2BE/wz8Y6D8Ste1nw1dWVnqep3%2Bs6XNC1rdSxrFqa3L2d3fedbvKsYceYUkmK17L%2B1B40/a%2B8GfGL4b%2BGfgv8AFL4babofxA8XyeHzb%2BKPhpqGp3dg8WharqjXHnQa1aJKGbTVjEflLtEpbe23BAPfKK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRWdZ2v7cuo%2BJr3wdY/tdfBKbVNOgimvrKP4FasXgSTdsLf8AFUcZ2k464wehGZcoxaTe5Mpwi0pO19F5vsvuPd6K8Z/4Qf8A4KF/9HQfBn/ww%2Brf/NRWP4xuP21vh/FYTeM/2xPgfpw1TUodP04XHwK1YNc3MrbY4kA8UZZifToAScAE1QqlSnSg5zaSXVuy7bv1X3o9%2Borxn/hB/wDgoX/0dB8Gf/DD6t/81FH/AAg//BQv/o6D4M/%2BGH1b/wCaigs9mor56vNZ/bIsPiJZfCS6/bO%2BBq%2BJdQsJb210UfAvVmna3jIDSlR4oOxcnALY3YOM7Tjf/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRTaa3IhUp1G1Fp2dnZ3s%2Bz7PVab6ns1FeM/8IP8A8FC/%2BjoPgz/4YfVv/mopl14Q/wCCgNlbSXt7%2B1T8FYYYULyyy/ArVVVFAyWJPinAAHOaRZ7TRXzj8K/GP7W/xv0q/wBe%2BEn7a/wM1%2Bw03U5NPur/AE74E6u8H2hFVmVJP%2BEo2ygB1%2BdCy5yM5BA6j/hB/wDgoX/0dB8Gf/DD6t/81FVKMoS5ZKzJhOFSClB3T6rVHs1FeM/8IP8A8FC/%2BjoPgz/4YfVv/moo/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRUlHs1FeM/8ACD/8FC/%2BjoPgz/4YfVv/AJqKP%2BEH/wCChf8A0dB8Gf8Aww%2Brf/NRQB7NRXjJ8Ef8FCgMn9qH4MgDqf8AhQ%2Brf/NRXL%2BIfG37UfhF2j8Wft/fs46Wy/eXUfhNewEfXf4sFVGE5u0Vf01E2luz6Oor5NuP2mfiLaOY7r/gql%2BybEw6rJ4CmU/r4uqzpv7QPxc1hxFpH/BT79la6YnAW2%2BHlw5J/wCA%2BLjWv1bEpX9nL/wGX%2BRPtaX8y%2B9f5n1TRXg/h%2BH9tzxbF5/hX9sv4D6mmM79P%2BCupTDH1TxUa0v%2BEH/4KF/9HQfBn/ww%2Brf/ADUVi04uzLTT2PZqK8Z/4Qf/AIKF/wDR0HwZ/wDDD6t/81FH/CD/APBQv/o6D4M/%2BGH1b/5qKQHs1FeM/wDCD/8ABQv/AKOg%2BDP/AIYfVv8A5qKP%2BEH/AOChf/R0HwZ/8MPq3/zUUAezUV4z/wAIP/wUL/6Og%2BDP/hh9W/8AmorlND8d/tXeI/idq/wX0f8Abf8AgRJ4r0NY31LQH%2BBmrx3KI8ayLIqP4oBlTa65dNyg/KSCCKqMZSTaV7bkSqQg0pNK%2Bi832XmfSNFeM/8ACD/8FC/%2BjoPgz/4YfVv/AJqKP%2BEH/wCChf8A0dB8Gf8Aww%2Brf/NRUlns1FeI6v4c/b70LSrrXNW/ap%2BDENrZ27z3Mx%2BA2rkJGilmbA8UZOACeKoeBJP22Pid4SsfHfw//bF%2BB2r6PqUIlstQsfgZqrxyr9f%2BEo4IOQVOCCCCAQRTs7X6Ee0pqp7O65rXtfW217b2vpc99orxn/hB/wDgoX/0dB8Gf/DD6t/81FZHjiX9tn4a%2BH28V%2BPf2wvghpWmpcQwyXt38CtWWNHlkWNAx/4Sj5QWYDJ4GckgAmkFSpTo03ObSitW27JLu29Ee%2B0V4T4E8d/tU%2BEv2qfDXwU%2BNfxJ%2BHnifRPE/wAPPEOu29x4T%2BHt9otza3OnX2iQIC8%2BsXySxOmqSkgIhBjQhsZB92oLCiiigAooooAKKKKACiiigArxn9qD/kt/7OP/AGWa%2B/8AUK8UV7NXz9%2B3P480f4ZeOvgB4312OWSCy%2BMV6Vgt0LSzyt4M8TpHEgHV3dlQD1YVM5wpwc5OyWrIqVIUabqTdkldvskekfGb4n6t4TSx8DfD%2ByivvF/iBmi0Wzl/1dug/wBZeT4%2B7DGDk92OFHJ4ufDT4eeHPgv4OnjudY8%2B4ld73xD4g1Bwsl7cEZknlYnCj0GcKoA7ZrN%2BCfw717Smvfil8S1STxf4jVWv1VtyabbDmKxiPZEB%2BYj77lmOeK5j4%2B%2BOLrxlb%2BIvhhpngy7uLbQXsbnUtUikt52t51aO7hZrJzunt/kXeTtz8yrnaWHHh6c60/rFVWf2V/Kv/kpdeytHo7%2BFVxDoU3j6699pqnB30Vm9bJ2lJK8n9lWjfSTPXtM1bStbs11HRtSt7u3f7k9rMsiN9GUkGuH0f4D2j/Gm%2B%2BOHj3xC%2Bv6lGDb%2BE7We3CW%2BgWhVQ6wpkgzSNuLzn5iuFAVQQec%2BBemeJrvxrbeNtE8A2OhaHqWiSNq9xpWpxy6dqs2YDa3NpEh3JlDLuMiqQNqfNgNXsddx14b2WbUIVa9P4ZXW9m0tJJO197rmTtLVXspBXjHxj/aB8Za144l/Zz/Zes7TU/GoiVvEGu3il9N8JW79Jrkj/W3DDJitgdzY3NtQcw/Gz4yePPHnj2X9l79ma/ji8SCFJPGPi9ohLbeErOQcHHSS9kXPlQ9vvvhRz6D8Fvgr4D%2BAvgaHwJ4A0%2BRIRK099fXcplutRunOZbq4lPMsznlmPsBgAAbxjGlFSmrt7L9X%2Bi6%2Bm%2BdWvXzGtLD4aXLCLtOa3v1hD%2B90lPVQ2V5/Bl/Af9nXwZ8CNOvLqwu7vWvEmtSCfxP4v1h/Mv8AVp8fekf%2BCNeiRLhEXgDqT6BRXgvxR/aT8f8AxJ8eX37O37G1rZ6h4hsJPI8XeO7%2BLzdH8JZ6o2Di7vcfdtlOFODIVAZaIQq4mbf3t7Jefb%2Bkkdn%2ByZZh404RstoxWrb306tvdtvvKUt2df8AHz9

In [4]:
# narrow valleys
plot_gd(x_scale=0.5, lr=0.1, steps=25)

<img src="%2BTl5ufo6erx8vP09fb3%2BPn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3%2BPn6/9oADAMBAAIRAxEAPwD9/KKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigBsssUETTzyKiIpZ3c4CgdST2Fcn8Jvj98CPj5ZX2pfAr41%2BEfGttplwINSuPCXiS11JLSUjIjla3kcRsQCdrYOK6m/lt4bGaa7haSJImaWNIWkLKAcgIoJckfwgEnoAa%2BTv2JfilafGD41eMf2ofGvwp%2BIXgvVPEun6T4V8NeDNd%2BEOv6U2k6HZ3F9JaPdzXFjHFJczS3VxLJ5btDaRNBEWDeZJKAfR/g742/Bn4ieL9d%2BHvw/wDi54Y13X/C04h8TaHo%2Bv291eaRISwCXUMbs9uxKsAJApyp9DXT18PfsKfD/wCIOl/GH4U%2BGNW%2BFHijQ9Q%2BE/wz8Y6D8Ste1nw1dWVnqep3%2Bs6XNC1rdSxrFqa3L2d3fedbvKsYceYUkmK17L%2B1B40/a%2B8GfGL4b%2BGfgv8AFL4babofxA8XyeHzb%2BKPhpqGp3dg8WharqjXHnQa1aJKGbTVjEflLtEpbe23BAPfKK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRXE%2BNfir%2B0z8PdYi8N%2BKv27fgLDqk8yxRaVb/A3V7i7Lk4A8mHxQ0g%2BpXH5Um1FaswxGJw2Ehz15qC7yaS/FrXy1fkfTlFeM/wDCD/8ABQv/AKOg%2BDP/AIYfVv8A5qKP%2BEH/AOChf/R0HwZ/8MPq3/zUUzc9morw6/0X9u7S72107U/2uvgdbXF85Sygn%2BCGqI9wwGSqKfFOXPsM1b/4Qf8A4KF/9HQfBn/ww%2Brf/NRQJSi20nsezUV4V4isP25PCGnNrHiz9sP4FaXaJ9%2B61H4J6nBGv1Z/FQArhfC37Qvx58d%2BPrb4Z%2BA/2%2BfgTr2sXbusUWh/ATW7yFdqM53zw%2BJmhjGFOCzjJwBkkCqUZSV0jmrY7BYepGnVqxjKTsk5JNt7WV7v5I%2Br6K8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRR/wg/8AwUL/AOjoPgz/AOGH1b/5qKk6j2aivGf%2BEH/4KF/9HQfBn/ww%2Brf/ADUUf8IP/wAFC/8Ao6D4M/8Ahh9W/wDmooA9morxn/hB/wDgoX/0dB8Gf/DD6t/81FH/AAg//BQv/o6D4M/%2BGH1b/wCaigD2aivGf%2BEH/wCChf8A0dB8Gf8Aww%2Brf/NRR/wg/wDwUL/6Og%2BDP/hh9W/%2BaigD2aivGH8E/wDBQiNS8n7UfwYUAck/AjVgB/5dNZGo3f7ZmkEjVv22fgDakdftHwa1FMf99eKhTSb2JlOEFeTS9T36ivmi8%2BKH7Renki//AOCif7NEGOvnfC27XH5%2BLKoTfHr4x25xcf8ABTf9lmPHXf8ADm4H8/FtWqNZ7Rf3P/IweMwcd6sV/wBvR/8Akj6mor5T/wCGifirnH/D0T9lP/w30/8A811SxfH34wT/AOo/4KdfssP/ALnw6uD/AO7dT9hX/kf3P/IX1/Av/l9D/wADj/8AJH1PRXzNafFX9oa/IFj/AMFGP2Zps9PK%2BF9238vFlbGn6n%2B2Jq5A0r9uD9n66J6fZ/g5qD5/LxWal06i3T%2B5/wCRpHEYefwzi/SSf6n0DRXi8Xgz/goLPGJYP2pfgu6noyfAnViD/wCXTTv%2BEH/4KF/9HQfBn/ww%2Brf/ADUVBsezUV4z/wAIP/wUL/6Og%2BDP/hh9W/8Amoo/4Qf/AIKF/wDR0HwZ/wDDD6t/81FAHs1FeM/8IP8A8FC/%2BjoPgz/4YfVv/moo/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRQB7NRXjP/CD/wDBQv8A6Og%2BDP8A4YfVv/moo/4Qf/goX/0dB8Gf/DD6t/8ANRQB7NRXjP8Awg//AAUL/wCjoPgz/wCGH1b/AOaivPfiL8c/2ivhH4yTwH8UP26fgfoGoSwJNE2q/s/65DbOr5xi4bxL5OflOV35HcDIpxjKTslcwxOKwuDp%2B0rzUI3teTSV3srtpfifVFFeC/Dn4jftP6D%2B0/4W%2BEfxc%2BKXw48XeHfF3w41/wAQ2N94O8AXujzwzafe6HDF%2B8m1i%2BSaGSPVZCQEU5jQhiCQfeqRsmmroKKKKBhRRRQAUUUUAFFFFABXz/8Atwf8LCPxF/Z5T4WnR11tvjRdi1k14Sm1jB8GeJ97MIvnYhdxCgjJwCQOa%2BgK8Y/akdIvjX%2BznJI4VV%2BMt%2BWZjgAf8IT4o5oauiKsPaU3C7V1unZrzT1s/OxZ/wCGYfE/jr998f8A48%2BI/Ekb/wCs0PRZf7G0wr/caO2Pmyj3eU59K67w78OvhH8BPDF1feAvhpaaZb20BeaPQNEMt1cAdtsStLMx4/vE1yfiz9tj4I6Rrcvg7wDdan4%2B8QwnEmheAdObUpIz0/eypiCHB6%2BZIuPwr0nwfrOt%2BIfCljrniHwncaHf3Vsslzo93cxyyWjkcxs8TMjEeqkij2Ps/ea%2B/f8AzPEwFLI5YiX1RxnVitZ61GumtSXMr33iprzSOas/jnoerfD/AMTeNLDQdTtbnwtBO2paPq9oYLiKRLZbhUZRuADRvGwIJ4fnnIrym8%2BOV3458F6Np3iD4oyI914gsLnxBdaJpV5oippU6uim2meQvJB9oUKbmN/mAI4V1Ndl8KPhF4p8P%2BJte1W08H6X4U0HVgkcnhxr99Q%2B2P5kpnu5ApVIpZEaNRtZ%2BAS%2BTgA03wH%2Bzv8AC/xzB4J8UePra61LWtJfTNC8MeItRt38nT3l3tbQRFVdomkVRmQuSY1UNwFroioJuyv%2BJx1Y5vi6VNzkop3jK94pvm0la6fvJKNtHrKySaZ5T4g%2B13smrXfxR07U/Ea2Oi3/AIXsdfs9Hku7uHVLC6WS1kQQxlopp/P5dQqs9qm7GBX034Uh1vUPAem2/ix5E1GfSIV1JomMbrMYgJCCOVO7PI5FeO3f7THgf4ZeAdZ0X4XfB6fRrXS9O1Z/CKtZW0Gn6pPYzCK4SJIJd8eJWJCyLE0oV2Xj5j3Pwh%2BI3i/WPEmrfDn4i3OlXeq6bbW99Z6tolrJb22pWU4bZIsMkkrRFSpUr5j9VOTk1VWnU5LtWt/X/DmuU08Phq7XtOeU1Z6O17t6ydpPa0W1eyeuqMPw7%2Bwr%2ByzoeorruqfCu38R6mOX1TxjdzaxO7f3s3jyAH/dAr1PSNG0fQLBNK0HSbaytYhiO2tIFjjQeyqABXC/Gbx/%2B0D4Z1Wy0H4IfAK18Utd27Pcazq3iqHTrKwfdgK42STSHHzYRMY/iBrjR8NP2%2BPH/wA/jf8AaS8I%2BB4Dw1j8P/CJvZivcfatRdgD7iAfQUlTlUipTmkvN3f3K7/I6

In [5]:
# narrower valleys
plot_gd(x_scale=0.2, lr=0.1, steps=25)

<img src="%2BTl5ufo6erx8vP09fb3%2BPn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3%2BPn6/9oADAMBAAIRAxEAPwD9/KKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigBsssUETTzyKiIpZ3c4CgdST2Fcn8Jvj98CPj5ZX2pfAr41%2BEfGttplwINSuPCXiS11JLSUjIjla3kcRsQCdrYOK6m/lt4bGaa7haSJImaWNIWkLKAcgIoJckfwgEnoAa%2BTv2JfilafGD41eMf2ofGvwp%2BIXgvVPEun6T4V8NeDNd%2BEOv6U2k6HZ3F9JaPdzXFjHFJczS3VxLJ5btDaRNBEWDeZJKAfR/g742/Bn4ieL9d%2BHvw/wDi54Y13X/C04h8TaHo%2Bv291eaRISwCXUMbs9uxKsAJApyp9DXT18PfsKfD/wCIOl/GH4U%2BGNW%2BFHijQ9Q%2BE/wz8Y6D8Ste1nw1dWVnqep3%2Bs6XNC1rdSxrFqa3L2d3fedbvKsYceYUkmK17L%2B1B40/a%2B8GfGL4b%2BGfgv8AFL4babofxA8XyeHzb%2BKPhpqGp3dg8WharqjXHnQa1aJKGbTVjEflLtEpbe23BAPfKK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRR/wg/8AwUL/AOjoPgz/AOGH1b/5qKAPZqK8Z/4Qf/goX/0dB8Gf/DD6t/8ANRR/wg//AAUL/wCjoPgz/wCGH1b/AOaigD2aivFh4P8A%2BCgRnNsP2qPgt5gGTH/wonVd2PXH/CU1UfTv25o9fTwpJ%2B2F8C11SSAzR6afglqYuGjHVxH/AMJVuK%2B%2BMUWZLnCO7Xb59vXy3PdKK%2BfbTVv2wr9NWfT/ANuH9n64Gglhrn2f4N6i508gZInC%2BKj5RAGcNip/Bo/bb%2BIehR%2BJvA/7YnwN1XT5WZUu7L4G6q6Fh1GR4p4I9KdmuhnHEYeclGM0272Saezs9m9no%2Bz0dnoe90V4z/wg/wDwUL/6Og%2BDP/hh9W/%2Baij/AIQf/goX/wBHQfBn/wAMPq3/AM1FI2PZqK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRR/wg/8AwUL/AOjoPgz/AOGH1b/5qKAPZqK8Z/4Qf/goX/0dB8Gf/DD6t/8ANRR/wg//AAUL/wCjoPgz/wCGH1b/AOaigD2aivGf%2BEH/AOChf/R0HwZ/8MPq3/zUUf8ACD/8FC/%2BjoPgz/4YfVv/AJqKAPZqK8Wm8H/8FArcbrj9qj4LIPV/gTqo/n4pqnPYftyWv/Hz%2B2H8Co/%2BunwT1Mfz8VUEynCO7S%2BZ7rRXz/NqH7Y9v/x8ftu/s/x/7/wb1Afz8VVVl8VftXQf6/8Ab0/Z1T/f%2BEd8P/drp8suxk8VhY71I/8AgUf/AJI%2BiqK%2BbZPiD%2B0xD/rv%2BChH7Nqf73wqvR/7tlQP8VP2hov9Z/wUX/ZmX/e%2BF92P/dsp8k%2Bxm8fgFvWh/wCBx/8Akj6Zor5hf4x/HiP/AFn/AAUk/ZhX6/DO6H/u21G3xu%2BNyfe/4KW/suj6/Da5/wDmto5J9n9z/wAif7Sy1f8AL%2BH/AIHD/wCTPqKivlo/Hb4zr97/AIKafstD6/Di5/8Amto/4Xx8Zf8ApJt%2Byz/4bm4/%2Ba2n7OfZ/c/8hf2plf8A0EU//BkP/kz6lor5a/4Xx8Zf%2Bkmv7LX/AIbm4/8AmtpR8c/jU33f%2BCmP7LZ%2Bnw3uf/mto9nPs/uf%2BQf2pln/AEEU/wDwOH/yZ9SUV8vr8avjk/3P%2BClX7L5%2Bnw1uf/mtqRPi78fpP9X/AMFH/wBmNvp8Mbo/%2B7bS5J9n9zGsyy17V4f%2BBw/%2BTPpyivmmP4m/tGzf6r/goj%2BzS3%2B78Lbw/wDu2VYi8cftQzf6n/goB%2Bzg/wDu/Ce9P/u2Ucsuxax2Be1WH/gUf/kj6Oor55h1/wDa4uP9R%2B3X%2Bzy/%2B58IL8/y8V1bhf8AbTuf%2BPf9tL4CSf7nwY1E/wAvFVLll2NFiMPLacX81/me90V4T4E8d/tU%2BEv2qfDXwV%2BNXxJ%2BHnifRPE/w88Q67b3HhP4e32i3Nrc6dfaJAgLz6xfJLE6apKSAiEGNCGxkH3akbBRRRQAUUUUAFFFFABRRRQAV8%2Bftz/EnwX8JPiD%2Bz146%2BIOsmw0u1%2BNN2s9yLaWYqX8GeJ0UBIlZzlmA4B619B14z%2B1B/yW/wDZx/7LNff%2BoV4opq19SKiqOm1TaUraNq6T81dX9Lr1G/8ADe/7Pt5x4ai8aa0x6LpPw51iTP0JtQD%2BdH/DaNlcc6P%2BzB8Z70H7pX4fS24b8bl469ooJAGScAdSavmpfy/j/wAA8v6rnUvixUV6Uo/%2B3TZ4uP2pvixqX/It/sP/ABLlz93%2B05tKsvz33hxXqHgHXvE3ifwjZ674x8DXHhrUrhXNzol3ew3Mlth2VQ0kDNG2VAb5ScbsHkGmaz8TPhv4dz/wkHxB0Sx2/e%2B2atDFj/vphV/QPEXh/wAWaRD4g8La7Z6lYXAY299p90k0MuGKna6Eq2GBBweoIol8N%2BW33/qbYSlXp1n7TEuo7fDanG3naOvlq7HhnjWf4baH%2B1XofiXwtrHhY6hbXNyPE2l6JpaRarCDZymS7vrhHJkt1UR/u5EXkqwdjtWs/wAVaXPcfFq50q30S8l8T6j8RtI1jQdXj06RlbRFjgE3%2BkbdiJGqXWY2YZ3KACWFehaj45%2BKfhr4/wCieBtS13QtU0rxEL2WPSrDR5YrzS7SCLctzLMZ3WRTIUiP7uMbpFAJPBw9Y%2BI3xK8KftIXNj471rxDp/g24vLCz8ONbaBZvp13czoU8ma5O64WRpyqrgKgLKC3PHRGMna3a54lbB0XzczsnV1tHRNpXeraV0k1LpJ7a2PNn%2BGXjbxR4RXwloHw21SHUvDnga/0/W0vNMkt4dQuk1CKeKCGaQLHcCYpLKHjZlXcA5VmxXtnwIt9S1HX/GPj1vD2paTpuu6zDJptlq1i9rcPst0WSdoXAZNz/KNwBIiz0IrxbWP2hfjb4X%2BGV18R7vx82oN4i8LazfW9k%2Bn26R6FNZ3OzMJjjVyiw5BEzSN5gB3AfLXsfwO1DxNovjvxj8Itb8Wajr1t4cewn0/V9YkR7pluondopHRVVijR5Hyg7ZF9qqtSqRg27fj31/H/AIBGVUMLTxlNwcr%2Bail8LUdm38Kd7aN6ys9CPx58JP2jfEvi291bwf8Ata3XhvSZ3U2ekWvgnT7lrYBFBHnThi%2BWBbJHG7Hasj/hmz4%2B3n/IZ/bq8cSZ6/YfD%2BjW3/oNocVe%2BIX7Qvxd8K%2BMr7wp4N/Y68b%2BJ4bR1WLWrPUtLtrS6yitmMz3SvgElTuUcqe3NY3/AA0H%2B2Jc82P/AAT91OM

In [6]:
# bigger learning rate then?
# x is changed faster, but y changes are too big, leads to oscillation
plot_gd(x_scale=0.2, lr=1.8, steps=25)

<img src="%2BTl5ufo6erx8vP09fb3%2BPn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3%2BPn6/9oADAMBAAIRAxEAPwD9/KKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigAooooAKKKKACiiigBsssUETTzyKiIpZ3c4CgdST2Fcn8Jvj98CPj5ZX2pfAr41%2BEfGttplwINSuPCXiS11JLSUjIjla3kcRsQCdrYOK6m/lt4bGaa7haSJImaWNIWkLKAcgIoJckfwgEnoAa%2BTv2JfilafGD41eMf2ofGvwp%2BIXgvVPEun6T4V8NeDNd%2BEOv6U2k6HZ3F9JaPdzXFjHFJczS3VxLJ5btDaRNBEWDeZJKAfR/g742/Bn4ieL9d%2BHvw/wDi54Y13X/C04h8TaHo%2Bv291eaRISwCXUMbs9uxKsAJApyp9DXT18PfsKfD/wCIOl/GH4U%2BGNW%2BFHijQ9Q%2BE/wz8Y6D8Ste1nw1dWVnqep3%2Bs6XNC1rdSxrFqa3L2d3fedbvKsYceYUkmK17L%2B1B40/a%2B8GfGL4b%2BGfgv8AFL4babofxA8XyeHzb%2BKPhpqGp3dg8WharqjXHnQa1aJKGbTVjEflLtEpbe23BAPfKK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRR/wg/8AwUL/AOjoPgz/AOGH1b/5qKAPZqK8Z/4Qf/goX/0dB8Gf/DD6t/8ANRR/wg//AAUL/wCjoPgz/wCGH1b/AOaigD2aivFh4P8A%2BCgRnNsP2qPgt5gGTH/wonVd2PXH/CU1UfTv25o9fTwpJ%2B2F8C11SSAzR6afglqYuGjHVxH/AMJVuK%2B%2BMUWZLnCO7Xb59vXy3PdKK%2BfbTVv2wr9NWfT/ANuH9n64Gglhrn2f4N6i508gZInC%2BKj5RAGcNip/Bo/bb%2BIehR%2BJvA/7YnwN1XT5WZUu7L4G6q6Fh1GR4p4I9KdmuhnHEYeclGM0272Saezs9m9no%2Bz0dnoe90V4z/wg/wDwUL/6Og%2BDP/hh9W/%2Baij/AIQf/goX/wBHQfBn/wAMPq3/AM1FI2PZqK8Z/wCEH/4KF/8AR0HwZ/8ADD6t/wDNRR/wg/8AwUL/AOjoPgz/AOGH1b/5qKAPZqK8Z/4Qf/goX/0dB8Gf/DD6t/8ANRR/wg//AAUL/wCjoPgz/wCGH1b/AOaigD2aivGf%2BEH/AOChf/R0HwZ/8MPq3/zUUf8ACD/8FC/%2BjoPgz/4YfVv/AJqKAPZqK8Wm8H/8FArcbrj9qj4LIPV/gTqo/n4pqnPYftyWv/Hz%2B2H8Co/%2BunwT1Mfz8VUEynCO7S%2BZ7rRXz/NqH7Y9v/x8ftu/s/x/7/wb1Afz8VVVl8VftXQf6/8Ab0/Z1T/f%2BEd8P/drp8suxk8VhY71I/8AgUf/AJI%2BiqK%2BbZPiD%2B0xD/rv%2BChH7Nqf73wqvR/7tlQP8VP2hov9Z/wUX/ZmX/e%2BF92P/dsp8k%2Bxm8fgFvWh/wCBx/8Akj6Zor5hf4x/HiP/AFn/AAUk/ZhX6/DO6H/u21G3xu%2BNyfe/4KW/suj6/Da5/wDmto5J9n9z/wAif7Sy1f8AL%2BH/AIHD/wCTPqKivlo/Hb4zr97/AIKafstD6/Di5/8Amto/4Xx8Zf8ApJt%2Byz/4bm4/%2Ba2n7OfZ/c/8hf2plf8A0EU//BkP/kz6lor5a/4Xx8Zf%2Bkmv7LX/AIbm4/8AmtpR8c/jU33f%2BCmP7LZ%2Bnw3uf/mto9nPs/uf%2BQf2pln/AEEU/wDwOH/yZ9SUV8vr8avjk/3P%2BClX7L5%2Bnw1uf/mtqRPi78fpP9X/AMFH/wBmNvp8Mbo/%2B7bS5J9n9zGsyy17V4f%2BBw/%2BTPpyivmmP4m/tGzf6r/goj%2BzS3%2B78Lbw/wDu2VYi8cftQzf6n/goB%2Bzg/wDu/Ce9P/u2Ucsuxax2Be1WH/gUf/kj6Oor55h1/wDa4uP9R%2B3X%2Bzy/%2B58IL8/y8V1bhf8AbTuf%2BPf9tL4CSf7nwY1E/wAvFVLll2NFiMPLacX81/me90V4T4E8d/tU%2BEv2qfDXwV%2BNXxJ%2BHnifRPE/w88Q67b3HhP4e32i3Nrc6dfaJAgLz6xfJLE6apKSAiEGNCGxkH3akbBRRRQAUUUUAFFFFABRRRQAV8%2Bftz/EnwX8JPiD%2Bz146%2BIOsmw0u1%2BNN2s9yLaWYqX8GeJ0UBIlZzlmA4B619B14z%2B1B/yW/wDZx/7LNff%2BoV4opq19SKiqOm1TaUraNq6T81dX9Lr1G/8ADe/7Pt5x4ai8aa0x6LpPw51iTP0JtQD%2BdH/DaNlcc6P%2BzB8Z70H7pX4fS24b8bl469ooJAGScAdSavmpfy/j/wAA8v6rnUvixUV6Uo/%2B3TZ4uP2pvixqX/It/sP/ABLlz93%2B05tKsvz33hxXqHgHXvE3ifwjZ674x8DXHhrUrhXNzol3ew3Mlth2VQ0kDNG2VAb5ScbsHkGmaz8TPhv4dz/wkHxB0Sx2/e%2B2atDFj/vphV/QPEXh/wAWaRD4g8La7Z6lYXAY299p90k0MuGKna6Eq2GBBweoIol8N%2BW33/qbYSlXp1n7TEuo7fDanG3naOvlq7HhnjWf4baH%2B1XofiXwtrHhY6hbXNyPE2l6JpaRarCDZymS7vrhHJkt1UR/u5EXkqwdjtWs/wAVaXPcfFq50q30S8l8T6j8RtI1jQdXj06RlbRFjgE3%2BkbdiJGqXWY2YZ3KACWFehaj45%2BKfhr4/wCieBtS13QtU0rxEL2WPSrDR5YrzS7SCLctzLMZ3WRTIUiP7uMbpFAJPBw9Y%2BI3xK8KftIXNj471rxDp/g24vLCz8ONbaBZvp13czoU8ma5O64WRpyqrgKgLKC3PHRGMna3a54lbB0XzczsnV1tHRNpXeraV0k1LpJ7a2PNn%2BGXjbxR4RXwloHw21SHUvDnga/0/W0vNMkt4dQuk1CKeKCGaQLHcCYpLKHjZlXcA5VmxXtnwIt9S1HX/GPj1vD2paTpuu6zDJptlq1i9rcPst0WSdoXAZNz/KNwBIiz0IrxbWP2hfjb4X%2BGV18R7vx82oN4i8LazfW9k%2Bn26R6FNZ3OzMJjjVyiw5BEzSN5gB3AfLXsfwO1DxNovjvxj8Itb8Wajr1t4cewn0/V9YkR7pluondopHRVVijR5Hyg7ZF9qqtSqRg27fj31/H/AIBGVUMLTxlNwcr%2Bail8LUdm38Kd7aN6ys9CPx58JP2jfEvi291bwf8Ata3XhvSZ3U2ekWvgnT7lrYBFBHnThi%2BWBbJHG7Hasj/hmz4%2B3n/IZ/bq8cSZ6/YfD%2BjW3/oNocVe%2BIX7Qvxd8K%2BMr7wp4N/Y68b%2BJ4bR1WLWrPUtLtrS6yitmMz3SvgElTuUcqe3NY3/AA0H%2B2Jc82P/AAT91OM